In [38]:
import numpy as np
import matplotlib.pyplot as plt 
from scipy import integrate

def gen_latex_table(table,energies):
    print(r"\begin{table}[]" + "\n" + r"\resizebox{10.0cm}{!}{" + "\n" + r"\begin{tabular}{|c|    c|c|c|c|c|c|c|c|c|c|c|c|c|c|c|}")
    print(r"\cline{3-15}")
    print(r"\multicolumn{1}{c}{}  & \multicolumn{1}{c}{}  & \multicolumn{13}{|c|}{Projectile Kinetic Energy (MeV)} \\")
    print(r"\cline{3-15}")
    header = "\multicolumn{1}{c}{}  & \multicolumn{1}{c}{}  & \multicolumn{1}{|c|}{4.0 MeV}"
    for i in energies:
        if (i==4.0):
            pass
        else:
            header += r" & ${0} \ MeV$".format(i)
    print(header + r" \\" )
    print(r"\cline{1-15}")
    print(r"\multirow{10}{*}{\rotatebox[origin=c]{90}{Projectile}}")
    for j in table:
        #j = j.replace("$1000.0^{\circ}$", "-")
        #j = j.replace("$-100.0\%$","-")
        #j = j.replace("$-1.0E+07$","-")
        #j = j.replace("E","x10^{")
        #j = j.replace("$-10000.0$", "-")
        print(j)
    print("\hline\n \end{tabular} \n } \n \end{table}")
    

def format_table(file,init,end):
    lines_data = []
    for i, line in enumerate(file):
        if i in np.arange(init, end): 
            lines_data.append(line) 
    data_array = np.loadtxt(lines_data)
    return data_array

def disp_vacancyCalc(file_path,n_files):
    """ Calculating the DPA throug the vacancy.txt """
    vacancy_ion_total = 0
    for i in range(n_files):
        file_path = file_path.replace("TRIM_outputs","TRIM_outputs_{0}".format(i+1))
        #print(file_path)
        vacancy_file = open(file_path + "VACANCY.txt", 'r')
        repCol_file = open(file_path + "NOVAC.txt", 'r')
        vacancy_array = format_table(vacancy_file,init=34,end=134)
        repCol_array = format_table(repCol_file,init=29,end=129)
        #target_displacement = np.add(vacancy_array[:,5], repCol_array[:,1])
        target_displacement= vacancy_array[:,5]
        dpa_depth = dpa_depth_calc(target_displacement)
       # plot(vacancy_array[:,0],target_displacement,file_path)
        #dpa = integral_calc(dpa_depth, vacancy_array[:,0])
        vacancy_ion = integral_calc(vacancy_array[:,5], vacancy_array[:,0])
        #return dpa
        vacancy_ion_total += vacancy_ion
    vacancy_mean = vacancy_ion_total/n_files
    vacancy_mean = round(vacancy_mean,2)
    if vacancy_mean == 0:
        flux_max = 0
    else:
        flux_max = doping_concentration/vacancy_mean # ions/cm2
    return vacancy_mean, flux_max
        
def disp_energyMet2(file_path,n_files): # REVIEW
    """ Calculating the DPA through the T_dam = E_phr - E_phi """
    vacancy_ion_total = 0
    for i in range(n_files):
        file_path = file_path.replace("TRIM_outputs","TRIM_outputs_{0}".format(i+1))
        phonons_file = open(file_path + "PHONON.txt", 'r')
        phonons_array = format_table(phonons_file, init=27,end=137)
        E_phr = integral_calc(phonons_array[:,2],phonons_array[:,0])
        E_phi = integral_calc(phonons_array[:,1],phonons_array[:,0])
        T_dam = abs(E_phr + E_phi)
        vacancy_ion = disp_depth_calc(T_dam)
        vacancy_ion_total +=  vacancy_ion
    
    vacancy_mean = vacancy_ion_total/n_files
    if vacancy_mean == 0:
        flux_max = 0
    else:
        flux_max = doping_concentration/vacancy_mean # ions/cm2
    return vacancy_mean, flux_max
    
def disp_depth_calc(T_dam,E_d=33):
    vacancy_ion = (0.8*(T_dam))/(2*E_d)
    '''
    i = 0
    T_dam = list(T_dam)
    while(i < len(T_dam)):
        T_dam[i] = (0.8*abs(T_dam[i]))/(2*E_d)
        i += 1
    '''
    return vacancy_ion

def dpa_depth_calc(y):
    i = 0
    y = list(y)
    while(i < len(y)):
        y[i] = (y[i]*1e8)/(si_atomic_density)
        i += 1
    return y
    
    
def integral_calc(y,x):
    # Get only data from 10um to 34 um.
    i = 0
    new_x = []
    new_y = []
    dp = 0
    while(i < len(x)):
        if (x[i] > 1e5) and (x[i] <= 3.4e5): # get the epitaxial layer region
            #print(yeap)
            new_x.append(x[i])
            dp += y[i]
            new_y.append(y[i])
        i += 1
    #now integrate 
    dpa = integrate.simpson(new_y,new_x)
    return dpa    


ions = ["H", "He", "Li" ,"Be", "B", "C", "N", "O", "F", "Ne"]
#ions = ["H", "He", "Li"]
energies = [4.0,8.0,10.0,15.0,20.0,30.0,40.0,50.0,60.0,70.0,80.0,90.0,100.0] # MeV

#ions = ["Be"]
#energies = [30.0] # MeV
nEvents=100000

table_dpaVac = []
table_dpaEn2 = []
table_fluxVac = []
table_flux_dpaEn2 = []

si_atomic_density = 4.977E22 #atoms/cm3
doping_concentration = 2.5E10 # n/cm2

def plot(x,y,name):
    plt.figure()
    plt.plot(x,y)
    plt.scatter(x,y)
    plt.xlabel("depth (Angstrom)")
    name = name.replace("TRIM_outputs/","")
    name = name.replace("/","")
    plt.ylabel("Vacancies (Number/Angstron.ion)")
   # plt.ylim(0,0.0002)
   # plt.xlim(1e5,5e5)
    plt.savefig("plots/" + name + ".pdf")
    
for ion in ions:
    line_dpaVac = "& " + ion
    line_dpaEn2 = "& " + ion
    line_fluxVac = "& " + ion
    line_flux_dpaEn2 = "& " + ion
    for energy in energies:
        if (ion=="H" or ion=="He" or ion=="Li"):
            nEvents=100000
        
        file_path = "TRIM_outputs/{0}_{1}MeV_n{2}/".format(ion,energy,nEvents)
        try:
            
            dpa_vacancy,flux_max_vac = disp_vacancyCalc(file_path,1)
            dpa_energy2,flux_max_dpaEn2 = disp_energyMet2(file_path,1)
            line_dpaVac += " & {0:.2f}".format(dpa_vacancy)
            line_fluxVac += " & {0:.1E}".format(flux_max_vac)
            line_dpaEn2 += " & {0:.2f}".format(dpa_energy2)
            line_flux_dpaEn2 += " & {0:.1E}".format(flux_max_dpaEn2)
            #print("{0} displacements/ion".format(dpa_vacancy))
        except:
            print("Not found data for {0} {1} MeV".format(ion,energy))
            pass
    line_dpaVac += r" \\"
    line_dpaEn2 += r" \\"
    line_fluxVac += r" \\"
    line_flux_dpaEn2 += r" \\"
    table_dpaVac.append(line_dpaVac)
    table_dpaEn2.append(line_dpaEn2)
    table_fluxVac.append(line_fluxVac)
    table_flux_dpaEn2.append(line_flux_dpaEn2)
    
print("------------------Tables-------------------")
print("------------------Vacancies/ion - VACANCY.txt-------------------") 
gen_latex_table(table_dpaVac,energies)
print("------------------Flux Max - VACANCY.txt-------------------")
gen_latex_table(table_fluxVac,energies)

print("------------------DPA - ENERGY PHONONS ----------------------")
gen_latex_table(table_dpaEn2,energies)
print("------------------Flux Max - ENERGY PHONONS ----------------------")
gen_latex_table(table_flux_dpaEn2,energies)

'''
print("------------------DPA - ENERGY METHOD 1-------------------")
gen_latex_table(table_dpaEn1,energies)
print("------------------DPA - ENERGY PHONONS ----------------------")
gen_latex_table(table_dpaEn2,energies)

#print("------------------DPA - ENERGY PHONONS ----------------------")
#gen_latex_table(table_dpaEn2,energies)
'''
print("End of Program")
        

------------------Tables-------------------
------------------Vacancies/ion - VACANCY.txt-------------------
\begin{table}[]
\resizebox{10.0cm}{!}{
\begin{tabular}{|c|    c|c|c|c|c|c|c|c|c|c|c|c|c|c|c|}
\cline{3-15}
\multicolumn{1}{c}{}  & \multicolumn{1}{c}{}  & \multicolumn{13}{|c|}{Projectile Kinetic Energy (MeV)} \\
\cline{3-15}
\multicolumn{1}{c}{}  & \multicolumn{1}{c}{}  & \multicolumn{1}{|c|}{4.0 MeV} & $8.0 \ MeV$ & $10.0 \ MeV$ & $15.0 \ MeV$ & $20.0 \ MeV$ & $30.0 \ MeV$ & $40.0 \ MeV$ & $50.0 \ MeV$ & $60.0 \ MeV$ & $70.0 \ MeV$ & $80.0 \ MeV$ & $90.0 \ MeV$ & $100.0 \ MeV$ \\
\cline{1-15}
\multirow{10}{*}{\rotatebox[origin=c]{90}{Projectile}}
& H & 1.57 & 0.70 & 0.49 & 0.33 & 0.21 & 0.14 & 0.09 & 0.06 & 0.06 & 0.05 & 0.04 & 0.03 & 0.04 \\
& He & 114.87 & 18.73 & 12.49 & 7.00 & 5.25 & 3.25 & 2.43 & 1.98 & 1.64 & 1.28 & 1.17 & 1.02 & 0.87 \\
& Li & 0.00 & 289.77 & 308.67 & 42.68 & 24.59 & 13.89 & 9.82 & 7.65 & 6.21 & 5.44 & 4.64 & 4.42 & 3.83 \\
& Be & 0.00 & 394.32 & 457.24